In [ ]:
% pip install -r./ requirements.txt

In [ ]:
% pip install -r./ yolov5 / requirements.txt

In [ ]:
!python yolov5 / train.py --img 640 --batch 16 --epochs 2 --data yolov5 / data / coco128.yaml --weights pretrained_models / yolov5n.pt --cache ram --device cpu --project./ trained_models --name xd

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('datasets/test_db/annotations.csv')

In [12]:
df

,filename,width,height,label,xmin,ymin,xmax,ymax
0,0.jpg,416,416,black-bishop,280,227,310,284
1,0.jpg,416,416,black-king,311,110,345,195
2,0.jpg,416,416,black-queen,237,85,262,159
3,0.jpg,416,416,black-rook,331,277,366,333
4,0.jpg,416,416,black-rook,235,3,255,51
...,...,...,...,...,...,...,...,...
104,6.jpg,416,416,black-rook,335,297,368,355
105,6.jpg,416,416,black-queen,234,97,260,173
106,6.jpg,416,416,black-pawn,273,84,291,128
107,6.jpg,416,416,black-pawn,204,118,223,162


In [13]:
grouped_df = df.groupby(['label']).agg({
    'xmin': list,
    'ymin': list,
    'xmax': list,
    'ymax': list,
    'filename': list
})

In [30]:
grouped_df

,xmin,ymin,xmax,ymax,filename
label,,,,,
black-bishop,"[280, 298, 207, 251, 236, 236, 135, 174]","[227, 38, 109, 241, 110, 35, 195, 153]","[310, 318, 227, 279, 260, 262, 164, 196]","[284, 93, 163, 306, 169, 97, 253, 210]","[0.jpg, 1.jpg, 1.jpg, 4.jpg, 4.jpg, 4.jpg, 4.j..."
black-king,"[311, 243, 302, 257]","[110, 172, 81, 268]","[345, 277, 336, 291]","[195, 256, 167, 354]","[0.jpg, 2.jpg, 4.jpg, 6.jpg]"
black-knight,"[235, 172, 306, 325, 240]","[27, 134, 85, 199, 115]","[256, 196, 336, 354, 264]","[84, 195, 163, 255, 172]","[0.jpg, 0.jpg, 1.jpg, 1.jpg, 1.jpg]"
black-pawn,"[267, 271, 280, 283, 254, 206, 207, 275, 264, ...","[38, 72, 145, 188, 284, 5, 103, 121, 18, 53, 8...","[286, 291, 303, 305, 278, 225, 226, 293, 281, ...","[82, 116, 190, 233, 333, 50, 147, 164, 59, 95,...","[0.jpg, 0.jpg, 0.jpg, 0.jpg, 0.jpg, 0.jpg, 0.j..."
black-queen,"[237, 284, 234, 203, 234]","[85, 137, 93, 139, 97]","[262, 305, 259, 230, 260]","[159, 206, 174, 214, 173]","[0.jpg, 1.jpg, 2.jpg, 4.jpg, 6.jpg]"
black-rook,"[331, 235, 339, 291, 333, 292, 320, 335]","[277, 3, 297, 10, 296, 10, 194, 297]","[366, 255, 369, 313, 367, 316, 350, 368]","[333, 51, 355, 59, 355, 61, 249, 355]","[0.jpg, 0.jpg, 1.jpg, 1.jpg, 2.jpg, 4.jpg, 4.j..."
white-bishop,"[52, 66, 46, 137, 66]","[174, 33, 246, 150, 34]","[79, 89, 73, 159, 90]","[231, 92, 305, 205, 92]","[0.jpg, 1.jpg, 1.jpg, 4.jpg, 4.jpg]"
white-king,"[56, 53, 129, 52, 131]","[111, 129, 128, 127, 129]","[87, 83, 161, 87, 162]","[191, 211, 213, 213, 213]","[0.jpg, 1.jpg, 2.jpg, 4.jpg, 6.jpg]"
white-knight,"[141, 52, 134, 83, 133]","[53, 221, 109, 298, 108]","[163, 83, 158, 115, 158]","[106, 281, 164, 360, 163]","[0.jpg, 0.jpg, 1.jpg, 4.jpg, 4.jpg]"


In [9]:
from collections import defaultdict
from pathlib import Path

from PIL import Image


# Sciezka do starych anotacji (labeli) w formacie csv
csv_path = 'datasets/test_db/annotations.csv'
# Sciezka do obrazow
images_path = 'datasets/test_db/images'
# Sciezka do nowych anotacji
labels_path = 'datasets/test_db_converted/labels'
# Sciezka do pliku z nazwami klas (yolo wymaga ID klasy, a nie nazwy!)
classes_names_path = 'datasets/test_db/classes.names'

classes_names_file = Path(classes_names_path)
classes_names_list = classes_names_file.read_text(encoding='utf8').split()
class_name_id_map = {class_name: idx for idx, class_name in enumerate(classes_names_list)}

images_dir = Path(images_path)
labels_dir = Path(labels_path)
labels_dir.mkdir(exist_ok=True, parents=True)

csv_file = Path(csv_path)
csv_content = csv_file.read_text(encoding='utf8')
csv_lines = csv_content.split()
csv_rows = [line.split(',') for line in csv_lines]
header_row = csv_rows[0]
# Mapowanie nazw kolumn na index (kolejność) na liście z nagłówkiem
col_idx = {col_name: idx for idx, col_name in enumerate(header_row)}
data_rows = csv_rows[1:]

file_bounds_map = defaultdict(list)

for row in data_rows:
    filename = str(row[col_idx['filename']])
    class_name = row[col_idx['label']]
    xmin = int(row[col_idx['xmin']])
    ymin = int(row[col_idx['ymin']])
    xmax = int(row[col_idx['xmax']])
    ymax = int(row[col_idx['ymax']])

    # Format yolo:
    # class x_center y_center width height

    image_file = images_dir / filename
    img_w, img_h = Image.open(image_file).size

    w = xmax - xmin
    h = ymax - ymin
    x_cnt = (xmin + xmax) / 2.0
    y_cnt = (ymin + ymax) / 2.0

    norm_bounds = [x_cnt / img_w, y_cnt / img_h, w / img_w, h / img_h]
    bounds_str = ' '.join(f'{i:.6f}' for i in norm_bounds)

    yolo_fmt_str = f'{class_name_id_map.get(class_name, 0)} {bounds_str}'
    file_bounds_map[filename].append(yolo_fmt_str)

for filename, bounds_list in file_bounds_map.items():
    image_file = images_dir / filename
    label_file_name = image_file.with_suffix('.txt').name
    label_file = labels_dir.joinpath(label_file_name)
    label_file_content = '\n'.join(bounds_list)
    label_file.write_text(label_file_content, encoding='utf8')
    # image_file.ren